# 1D Spatial Cross-Correlation

In [ ]:
########################################################################################
# Author: Justin Clancy, University of Melbourne, 2020                                 #
# - Pattern Matching -                                                                 #
# Measuring Spatial Cross-Correlation for 1D signal                                    #
########################################################################################

## The functions

### Import Modules Required

In [2]:
# Import Modules
from numpy import loadtxt
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

### Cross Correlation Function

In [ ]:
# Define a function to calculate the 1D spatial cross-correlation
def cor(f,g):
    # Creata an empty array to fill with the score at each m
    score_array = []
    # define the values of our variables noted earlier
    # length of pattern:
    n = len(f)
    # total number of offset positions we can have in the template
    m = np.arange(len(g)-(n-1))
    # loop through the amount of offsets we can work with
    for i in m:
        # Create an array filled with zeros equal to the size of n
        current_score = np.zeros(n)
        # loop through the amount of values in the pattern
        for j in np.arange(n):
            # multiply the pattern and template value at this position
            value = f[j]*g[j+i]
            # update this positions score in the zeros array
            current_score[j] = value
        # sum the values in this array and add this to our final array
        mcorr = np.sum(current_score)
        score_array.append(mcorr)
    # Return just the final score array
    return(score_array)


#-------Test---------#

# Test inputs
pattern = np.array([1,2,2,1])
template = np.array([0,1,1,0,1,2,2,1,0,0,0])

# Run test
score = cor(pattern,template)
print("Cross Correlation Scores: \n",score)

### Best lag & score

In [ ]:
# Define a function to locate the index of highest score (best lag)
def B_Lag(cscore):
    # Find the maximum value in the array
    max_score = np.max(cscore)
    # Find which index this occurs
    max_index = cscore.index(max_score)
    return(max_index,max_score)


#-----Test------#


max_loc_val = B_Lag(score)
print("Location of best lag:\n",max_loc_val[0],"\nScore of best lag: \n",max_loc_val[1])

### Array Energy Normalisation

In [ ]:
# Define a function to calculate array energy normalisation factor
# of two arrays
def arr_eng(f,g):
    # Check that the arrays are of the same length
    if len(f)==len(g):
        # Square and sum both arrays
        f_sum = np.sum(f**2)
        g_sum = np.sum(g**2)
        # Return the multiple of both sums and square root
        return(np.sqrt(f_sum*g_sum))
    else:
        # If the arrays are of different length, return an error
        return("Error: Input arrays must be the same size!")
  
#-------Test Inputs--------#


arr1 = np.array([0,1,1,2])
arr2 = np.array([1,1,0,2])

array_energy = arr_eng(arr1,arr2)
print("Array energy normalisation factor of two arrays:\n",array_energy)

### Edge matches (Padding)

In [ ]:
# Define function to zero-pad a template array
def pad(array,size):
    
    # Create an array of input size, full of zero values
    pading = np.zeros(size)
    # Add the input array into the center of the zero filled array
    right_pad = np.insert(array,len(array),pading)
    full_pad = np.insert(right_pad,0,pading)
    # Return padded array
    return(full_pad)


#-------Test inputs------#

# padding size: 3
array_to_pad = np.array([1,1,0,2])
# Expected return: [0,0,0,1,1,0,2,0,0,0]

padded_array = pad(array_to_pad,3)
print("Padded array is now:\n",padded_array)

## The Result

### Submission Question

In [ ]:
# We can now use the 1D spatial cross-correlation to find out the 
# distance between two microphones by the signal they receive from
# a speaker.

# NOTE: The sampling rate is 44.1KHz or 44100 samples per second

# Read the data files
sensor_1_data = loadtxt("/Users/justi/Desktop/sensor1Data.txt",skiprows=1)
sensor_2_data = loadtxt("/Users/justi/Desktop/sensor2Data.txt",skiprows=1)

# Combine the above functions to obtain the signal offset
def signal_corr(p,t):
    overlap = len(p)            # Length of pattern that will be iteratively moved
    
    # Pad the template array
    pad_t = pad(t,overlap-1)
    
    # Define an empty array to fill with results
    score_arr = []
    
    # Iterate the pattern across all possible matches over the template
    for i in np.arange(len(pad_t) - overlap + 1):
        # Note the current slice of template
        m = pad_t[i:i+overlap]
        # Calculate array energy for normalization
        energy = arr_eng(p,m)
        # Calculate normalized correlation score
        correlation = np.dot(p,m)/energy
        # Add results to the score array
        score_arr.append(correlation)
        
    # Remove the padding 
    score_og_size = score_arr[overlap - 1 : overlap + overlap + 1]
    
    # Record the maximum score and index location
    #max_score = np.max(score_arr)
    #best_lag = score_arr.index(max_score)
    best_lag,max_score = B_Lag(score_arr)
    
    return score_arr,max_score,best_lag-(overlap-1), score_og_size


## Run signal function

In [ ]:
# Run the function with the sensor data
start = timer()
result = signal_corr(sensor_1_data,sensor_2_data)
end = timer()
print("Runtime:", end-start,"s") # Runtime in seconds

### Plots and calculations

In [ ]:
# Calculate the time between signal being measured by each microphone
freq = 44100 # Hz
sound_speed = 333 # m/s
T = 1/freq
time = result[2]*T # s
print("Time between microphones hearing sound:",time,'s')

# Calculate distance between microphones
dist = sound_speed*time # m
print("Distance between microphones is:",dist,'m')

#--------Plots---------#

plt.plot(result[3])
plt.title('Spatial cross-correlation scores (with padding removed) for signal matching')
plt.xlabel('Index value')
plt.ylabel('Spatial cross-correlation score')
plt.show()

plt.plot(result[0])
plt.title('Spatial cross-correlation scores (with padding) for signal matching')
plt.xlabel('Index value')
plt.ylabel('Spatial cross-correlation score')
plt.show()
